- augmenter la RAM utilisée par SPARK !! : http://www.datastax.com/dev/blog/zen-art-spark-maintenance, http://www.datastax.com/dev/blog/common-spark-troubleshooting
- faire la table dans cassandra
- dropper le poids
- ajouter une colonne date (full timestamp)
- insérer donc fichier par fichier
- possible de paralleliser ?
- **faire un test sur la vitesse de chargement**

In [1]:
1+1

2

In [1]:
print(sc)

In [2]:
sc.master

u'spark://172.31.20.22:7077'

In [1]:
rdd = sc.parallelize([1, 2, 3]).map(lambda x: x*2).filter(lambda x: x > 2).collect()

In [2]:
rdd

[4, 6]

---

## Chargement des données

In [44]:
from os import listdir
from os.path import isfile, join
import re
from time import time
from pyspark.sql import SQLContext
sql = SQLContext(sc)

On récupère la liste des fichiers

In [8]:
directory = '/mnt/wikidata/wikistats/'
datafiles = [f for f in listdir(directory) 
             if isfile(join(directory, f)) and f.startswith('pagecounts')]
print('Il existe {} fichiers.'.format(len(datafiles)))

Il existe 2160 fichiers.


A chaque fichier on associer un datastamp :

In [36]:
def extract_date(datafile):
    rex = re.compile(r'pagecounts-(\d{4})(\d{2})(\d{2})-(\d{2}).*')
    s = rex.match(datafile)
    datetime = '{}-{}-{} {}:00'.format(*(s.group(i) for i in range(1, 5)))
    return datetime

In [38]:
datafiles_datetimes = [(datafile, extract_date(datafile)) for datafile in datafiles]

In [39]:
datafiles_datetimes[:10]

[('pagecounts-20110130-070000.gz', '2011-01-30 07:00'),
 ('pagecounts-20110305-210000.gz', '2011-03-05 21:00'),
 ('pagecounts-20110113-180000.gz', '2011-01-13 18:00'),
 ('pagecounts-20110313-220000.gz', '2011-03-13 22:00'),
 ('pagecounts-20110101-060000.gz', '2011-01-01 06:00'),
 ('pagecounts-20110314-110000.gz', '2011-03-14 11:00'),
 ('pagecounts-20110124-110000.gz', '2011-01-24 11:00'),
 ('pagecounts-20110116-220000.gz', '2011-01-16 22:00'),
 ('pagecounts-20110108-230000.gz', '2011-01-08 23:00'),
 ('pagecounts-20110109-060000.gz', '2011-01-09 06:00')]

On teste le premier fichier.

In [48]:
datafile = datafiles_datetimes[0][0]
datetime = datafiles_datetimes[0][1]
path='file:///mnt/wikidata/wikistats/' + datafile 

In [49]:
rdd = sc.textFile(path).map(lambda r: [datetime] + r.split(' '))
df = sql.createDataFrame(rdd, schema=['datetime', 'projectcode', 'page', 'views', 'weight'])

In [50]:
df.write.format("org.apache.spark.sql.cassandra").\
    options(table="wikipediadata", keyspace="projet").\
    save(mode="append")

On time l'insertion de 10 fichiers via le master:

In [ ]:
ts = [time()]
for datafile, datetime in datafiles_datetimes[1:11]:
    rdd = sc.textFile(path).map(lambda r: [datetime] + r.split(' '))
    df = sql.createDataFrame(rdd, schema=['datetime', 'projectcode', 'page', 'views', 'weight'])
    df.write.format("org.apache.spark.sql.cassandra").\
        options(table="wikipediadata", keyspace="projet").\
        save(mode="append")
    ts.append(time())

In [ ]:
for i, t in enumerate(ts[1:]):
    print('Chargement du fichier {} : {} secondes.'.format(i, t-ts[i-1]))
total = ts[-1] - ts[0]
m = total // 60
s = total % 60
print('Temps total pour 10 fichiers : {} minutes {} secondes'.format(m, s))

---

In [5]:
user = sql.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="training", table="user")

In [6]:
adults = user.where(user.age > 21)


In [7]:
adults.show()


+-------+---+--------------------+-------+
|user_id|age|      favorite_foods|   name|
+-------+---+--------------------+-------+
|      3|108|ArrayBuffer(Muffi...|Patrick|
|      1| 34|ArrayBuffer(Bacon...|    Jon|
|      2| 22|ArrayBuffer(Kale,...|   Dani|
+-------+---+--------------------+-------+

